## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sorong,ID,-0.8833,131.2500,79.03,84,100,1.03,overcast clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,64.99,76,42,1.99,scattered clouds
2,2,Cape Town,ZA,-33.9258,18.4232,66.99,77,0,1.14,clear sky
3,3,Huarmey,PE,-10.0681,-78.1522,71.02,82,94,8.63,overcast clouds
4,4,Victoria,HK,22.2855,114.1577,78.01,87,26,7.09,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sorong,ID,-0.8833,131.2500,79.03,84,100,1.03,overcast clouds
4,4,Victoria,HK,22.2855,114.1577,78.01,87,26,7.09,scattered clouds
8,8,Rikitea,PF,-23.1203,-134.9692,79.30,74,2,15.55,clear sky
13,13,Puerto Ayora,EC,-0.7393,-90.3518,79.00,91,33,4.00,scattered clouds
14,14,Vanimo,PG,-2.6741,141.3028,83.93,67,99,3.71,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
680,680,Bontang,ID,0.1333,117.5000,78.49,88,99,2.98,light rain
683,683,Tevaitoa,PF,-16.7833,-151.5000,82.31,73,38,20.60,light rain
684,684,Barra,BR,-11.0894,-43.1417,81.97,45,59,3.04,broken clouds
688,688,Dondo,MZ,-19.6094,34.7431,78.80,78,40,13.80,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sorong,ID,79.03,overcast clouds,-0.8833,131.2500,
4,Victoria,HK,78.01,scattered clouds,22.2855,114.1577,
8,Rikitea,PF,79.30,clear sky,-23.1203,-134.9692,
13,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,
14,Vanimo,PG,83.93,overcast clouds,-2.6741,141.3028,
...,...,...,...,...,...,...,...
680,Bontang,ID,78.49,light rain,0.1333,117.5000,
683,Tevaitoa,PF,82.31,light rain,-16.7833,-151.5000,
684,Barra,BR,81.97,broken clouds,-11.0894,-43.1417,
688,Dondo,MZ,78.80,scattered clouds,-19.6094,34.7431,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")    

In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
689,San Andres,CO,82.4,few clouds,12.5847,-81.7006,Hotel Arena Blanca


In [10]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))